In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,TensorDataset

In [2]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
pip install DeepMIMOv3 umap-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
#LWM을 하기위한 라이브러리 가져오기
import DeepMIMOv3
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt

plt . rcParams [ 'figure.figsize' ]  =  [ 12 ,  8 ]  # 기본 플롯 크기 설정

In [5]:
## Load and print the default parameters
# bandwith: 0.05GHz(50MHz 대역폭 사용)

parameters = DeepMIMOv3.default_params()
pprint(parameters)

{'OFDM': {'RX_filter': 0,
          'bandwidth': 0.05,
          'selected_subcarriers': array([0]),
          'subcarriers': 512},
 'OFDM_channels': 1,
 'active_BS': array([1]),
 'bs_antenna': {'FoV': array([360, 180]),
                'radiation_pattern': 'isotropic',
                'rotation': array([0, 0, 0]),
                'shape': array([8, 4]),
                'spacing': 0.5},
 'dataset_folder': './Raytracing_scenarios',
 'dynamic_scenario_scenes': array([1]),
 'enable_BS2BS': 1,
 'enable_doppler': 0,
 'enable_dual_polar': 0,
 'num_paths': 5,
 'scenario': 'O1_60',
 'ue_antenna': {'FoV': array([360, 180]),
                'radiation_pattern': 'isotropic',
                'rotation': array([0, 0, 0]),
                'shape': array([4, 2]),
                'spacing': 0.5},
 'user_rows': array([1]),
 'user_subsampling': 1}


In [6]:
## Change parameters for the setup
# Scenario O1_60 extracted at the dataset_folder
#LWM 동적 시나리오 불러오기
#자신의 LWM 파일 위치 경로 작성
# parameters['dataset_folder'] = r'/content/drive/MyDrive/Colab Notebooks/LWM'
parameters['dataset_folder'] = r'C:\Users\dlghd\졸업프로젝트\LWM'

# scnario = 02_dyn_3p5 <- 다운받은 파일(동적시나리오)
parameters['scenario'] = 'O2_dyn_3p5'
parameters['dynamic_scenario_scenes'] = np.arange(10) #scene 0~9

# 각 사용자-기지국 채널에 대해 최대 10개 멀티패스 경로 사용
parameters['num_paths'] = 10

# User rows 1-100
parameters['user_rows'] = np.arange(100)

# Activate only the first basestation
parameters['active_BS'] = np.array([1])

parameters['activate_OFDM'] = 1

parameters['OFDM']['bandwidth'] = 0.05 # 50 MHz
parameters['OFDM']['subcarriers'] = 512 # OFDM with 512 subcarriers
parameters['OFDM']['selected_subcarriers'] = np.arange(0, 64, 1)
#parameters['OFDM']['subcarriers_limit'] = 64 # Keep only first 64 subcarriers

parameters['ue_antenna']['shape'] = np.array([1, 1]) # Single antenna
parameters['bs_antenna']['shape'] = np.array([1, 32]) # ULA of 32 elements
#parameters['bs_antenna']['rotation'] = np.array([0, 30, 90]) # ULA of 32 elements
#parameters['ue_antenna']['rotation'] = np.array([[0, 30], [30, 60], [60, 90]]) # ULA of 32 elements
#parameters['ue_antenna']['radiation_pattern'] = 'isotropic'
#parameters['bs_antenna']['radiation_pattern'] = 'halfwave-dipole'

In [7]:
print(parameters)

{'dataset_folder': 'C:\\Users\\dlghd\\졸업프로젝트\\LWM', 'scenario': 'O2_dyn_3p5', 'dynamic_scenario_scenes': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), 'num_paths': 10, 'active_BS': array([1]), 'user_rows': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), 'user_subsampling': 1, 'bs_antenna': {'shape': array([ 1, 32]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'ue_antenna': {'shape': array([1, 1]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'enable_doppler': 0, 'enable_dual_polar'

In [8]:
## Generate and inspect the dataset
dataset = DeepMIMOv3.generate_data(parameters)

The following parameters seem unnecessary:
{'activate_OFDM'}

Scene 1/10

Basestation 1

UE-BS Channels


Generating channels: 100%|█████████████████████████████████████████████████████| 69040/69040 [00:25<00:00, 2758.37it/s]



BS-BS Channels


Generating channels: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 500.51it/s]



Scene 2/10

Basestation 1

UE-BS Channels


...

Generating channels: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 333.41it/s]



Scene 10/10

Basestation 1

UE-BS Channels


Generating channels: 100%|█████████████████████████████████████████████████████| 69040/69040 [00:22<00:00, 3090.53it/s]



BS-BS Channels


Generating channels: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 186.61it/s]


In [9]:
len(dataset)

10

In [10]:
dataset[0]

[{'user': {'paths': array([{'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
          {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
          {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
          ...,
          {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
          {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
          {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []}],
         dtype=object),
   'LoS': array([-1, -1, -1, ..., -1, -1, -1], dtype=int8),
   'location': array([[-91.03330231, -15.57

# 사용자 접근 데이터

In [11]:
user_data = dataset[0][0]['user']
print(user_data.keys())

dict_keys(['paths', 'LoS', 'location', 'distance', 'pathloss', 'channel'])


# 사용자 채널 정보 확인

In [12]:
# subcarries = 나눈 각각의 주파수 채널
# Channel = H <- 채널 벡터
# 채널 형태
# (user, UE antenna, Bs antenna, subcarrier)
channel = dataset[0][0]['user']['channel']
print(channel.shape)  

(69040, 1, 32, 64)


In [13]:
print(dataset[0][0]['user']['channel'][100])

[[[-4.9276509e-06+6.0179661e-07j -4.7681883e-06+1.3829425e-06j
   -4.4857170e-06+2.1285541e-06j ...  4.3711116e-06-2.4074948e-06j
    3.9297033e-06-3.0764131e-06j  3.3868639e-06-3.6660977e-06j]
  [-4.7825752e-06+1.6998159e-06j -4.4491662e-06+2.4436672e-06j
   -4.0009481e-06+3.1246111e-06j ...  3.8229477e-06-3.3767817e-06j
    3.2333687e-06-3.9455144e-06j  2.5602983e-06-4.4125736e-06j]
  [-4.3861578e-06+2.7614337e-06j -3.8878534e-06+3.4282084e-06j
   -3.2891933e-06+4.0066625e-06j ...  3.0543217e-06-4.2173128e-06j
    2.3400164e-06-4.6522073e-06j  1.5652473e-06-4.9671735e-06j]
  ...
  [-5.9705985e-06+1.8381670e-06j -5.5992200e-06+2.7702260e-06j
   -5.0834296e-06+3.6307945e-06j ...  4.8462462e-06-3.9304277e-06j
    4.1544481e-06-4.6554678e-06j  3.3555179e-06-5.2603964e-06j]
  [-5.3850690e-06+3.0986103e-06j -4.8194606e-06+3.9206407e-06j
   -4.1295657e-06+4.6415066e-06j ...  3.8328362e-06-4.8787911e-06j
    3.0023016e-06-5.4293314e-06j  2.0943648e-06-5.8398036e-06j]
  [-4.5357333e-06+4.1878

In [14]:
print(len(dataset[0][0]['user']['channel'][100]))

1


In [15]:
print(channel[10000][0][0])

[ 1.14151417e-05-4.36109121e-06j  1.13104134e-05-4.99916769e-06j
  1.11716581e-05-5.67118514e-06j  1.09873472e-05-6.37486755e-06j
  1.07464039e-05-7.10507175e-06j  1.04388646e-05-7.85394786e-06j
  1.00564839e-05-8.61123954e-06j  9.59326280e-06-9.36472679e-06j
  9.04586159e-06-1.01007672e-05j  8.41387282e-06-1.08049298e-05j
  7.69995040e-06-1.14626673e-05j  6.90977913e-06-1.20600052e-05j
  6.05189052e-06-1.25842089e-05j  5.13732675e-06-1.30243880e-05j
  4.17918318e-06-1.33720132e-05j  3.19203900e-06-1.36213148e-05j
  2.19131471e-06-1.37695433e-05j  1.19258925e-06-1.38170753e-05j
  2.10908027e-07-1.37673669e-05j -7.39877976e-07-1.36267463e-05j
 -1.64770893e-06-1.34040656e-05j -2.50281732e-06-1.31102279e-05j
 -3.29810814e-06-1.27576113e-05j -4.02939668e-06-1.23594264e-05j
 -4.69549514e-06-1.19290335e-05j -5.29813951e-06-1.14792647e-05j
 -5.84176632e-06-1.10217779e-05j -6.33314448e-06-1.05664840e-05j
 -6.78089191e-06-1.01210744e-05j -7.19489299e-06-9.69066787e-06j
 -7.58565420e-06-9.277610

# 사용자 위치 정보

In [16]:
location = dataset[0][0]['user']['location']
print(location.shape)      # (사용자 수, 3)
print(location[0:4])         # 첫 번째 사용자의 (x, y, z)

(69040, 3)
[[-91.03330231 -15.57629967   1.        ]
 [-90.83329773 -15.57629967   1.        ]
 [-90.63330078 -15.57629967   1.        ]
 [-90.4332962  -15.57629967   1.        ]]


# 경로정보

In [17]:
paths = dataset[0][0]['user']['paths']
#사용자 수
print(len(paths))
# 첫 번째 사용자 경로 정보
print(paths[0])

69040
{'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []}


# 기지국 정보

In [18]:
bs_data = dataset[0][0]['basestation']
print(bs_data.keys())


dict_keys(['paths', 'LoS', 'location', 'distance', 'pathloss', 'channel'])


# Scene 및 사용자 수

In [19]:
for i, scene in enumerate(dataset[0]):
    user_locs = scene['user']['location']
    print(f"Scene {i}: {len(user_locs)} users")

Scene 0: 69040 users


# 채널 수

In [20]:
len(dataset[0][0]['user']['channel'])

69040

In [21]:
print(dataset[0][0]['user']['paths'][0])

{'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []}


In [22]:
scene = dataset[0][0] # scene 0
ue_idx = 0 # 첫 번째 사용자
channel = scene['user']['channel'][ue_idx]
print(channel.shape)

(1, 32, 64)


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [24]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


# channel CIR mat 정보 가져오기

In [25]:
import scipy.io as sio

file_path = r'C:\Users\dlghd\졸업프로젝트\LWM\O2_dyn_3p5\scene_0\O2_dyn_3p5.1.CIR.mat'
mat_data = sio.loadmat(file_path)

# 파일 안의 key 확인
print(mat_data.keys())




dict_keys(['__header__', '__version__', '__globals__', 'CIR_array_full'])


In [26]:
# 일반적으로 CIR key는 'CIR' 또는 'cir' 같은 이름일 가능성 높음
H_cir = mat_data['__header__']  
print(H_cir)

b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Jun 30 11:33:01 2021'


# Time-Prediction 시작
## Time Series 형태로 변환
### 단일사용자 채널 예측

In [27]:
# H = dataset[0][0]['user']['channel'][100]  # (1, 32, 64), complex
# H = H.squeeze(0)  # → (32, 64)

# # 채널은 복소수 형태
# H_real = H.real   # shape: (32, 64)
# H_imag = H.imag   # shape: (32, 64)

# H_combined = np.concatenate([H_real, H_imag], axis=0)  # shape: (64, 64)


In [28]:
channel_sequence = []

for scene in dataset:
    H = scene[0]['user']['channel'][100]
    H = H.squeeze(0)
    # 채널은 복소수 형태 
    H_real = H.real # (32, 64) = (BS antenna, subcarrier)
    H_imag = H.imag
    H_concat = np.concatenate([H_real, H_imag], axis=0) 
    channel_sequence.append(H_concat)

# T는 sence
channel_sequence = np.stack(channel_sequence)  # shape: (T, 64, 64) = (real/imag쌍, subcarrier)

#서브케리어를 각가의 데이터로 보고 데이터 예측 
# 주파수 64개 10scene 있고 


In [33]:
# 데이터 만들기
# seq_len = 시퀀스를 잘라서 사용
# [0,1,2,3,4], [1,2,3,4,5], [2,3,4,5,6],[3,4,5,6,7],[4,5,6,7,8],[5,6,7,8,9]
seq_len = 5
X = []
y = []

for i in range(len(channel_sequence) - seq_len):
    input_seq = channel_sequence[i:i+seq_len-1] # t=0~3
    target = channel_sequence[i+seq_len-1] # t = 4
    
    X.append(input_seq)
    y.append(target)

# T = scene case
# samples = T - seq_len + 1
X = np.array(X) # shape: (samples, seq_len-1, 64, 64)
y = np.array(y) # shape: (samples, 64, 64)

In [34]:
print(len(channel_sequence)-seq_len)

5


In [35]:
# Flatten
# 4096 = 64 * 64
X_flat = X.reshape(X.shape[0], X.shape[1], -1) # (5,4,4096) = (샘플, 시퀀스 길이)
y_flat = y.reshape(y.shape[0],-1) #(5,4096) # 샘플 대상 예측

In [36]:
print(X_flat.shape)
print(y_flat.shape)

(5, 4, 4096)
(5, 4096)


In [37]:
# 데이터 생성
# 데이터 GPU로 계산하게 바꾸기
X_t = torch.tensor(X_flat, dtype = torch.float32)
y_t = torch.tensor(y_flat, dtype = torch.float32)
masked_pos = torch.full((X_t.shape[0], 1), seq_len - 2, dtype=torch.long, device='cuda')  # 마지막 위치 예측
X_t = X_t.to(device)
y_t = y_t.to(device)

In [44]:
# 데이터 설정

batch_size = 64

# 전체 데이터셋
full_dataset = TensorDataset(X_t, y_t, masked_pos)

# 비율 정하기
total_len = len(full_dataset)
train_len = int(total_len * 0.6)
val_len = int(total_len*0.2)
test_len = total_len - (train_len + val_len)

# 데이터 분리
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    full_dataset, [train_len, val_len, test_len])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [45]:
# 모델 초기화 및 gpu 이도 
# 손실 함수 및 옵티마이저 설정
from lwm_model import lwm

element_length = 64*64 # H_real * H_imag

model = lwm(element_length=element_length).to(device)
criterion =nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [52]:
# 모델학습
import time
start = time.time()

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch, masked_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        masked_batch = masked_batch.to(device)

        optimizer.zero_grad()
        pred, _ = model(X_batch, masked_batch)  # ← masked_pos 사용
        pred = pred.squeeze(1)  # (batch, 1, 4096) → (batch, 4096)

        loss = criterion(pred, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # 🔍 검증 루프 (Validation)
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_val, y_val, masked_val in val_loader:
            X_val = X_val.to(device)
            y_val = y_val.to(device)
            masked_val = masked_val.to(device)

            val_pred, _ = model(X_val, masked_val)
            val_pred = val_pred.squeeze(1)

            loss = criterion(val_pred, y_val)
            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} |  Train Loss: {train_loss/len(train_loader):.6f} |  Val Loss: {val_loss/len(val_loader):.6f}")

end = time.time()
print(f"\n⏱️ 전체 학습 시간: {round(end - start, 2)}초")


Epoch 1/100 |  Train Loss: 0.000612 |  Val Loss: 0.000064
Epoch 2/100 |  Train Loss: 0.001122 |  Val Loss: 0.000059
Epoch 3/100 |  Train Loss: 0.000427 |  Val Loss: 0.000059
Epoch 4/100 |  Train Loss: 0.000359 |  Val Loss: 0.000059
Epoch 5/100 |  Train Loss: 0.000604 |  Val Loss: 0.000061
Epoch 6/100 |  Train Loss: 0.000629 |  Val Loss: 0.000059
Epoch 7/100 |  Train Loss: 0.000580 |  Val Loss: 0.000053
Epoch 8/100 |  Train Loss: 0.000577 |  Val Loss: 0.000041
Epoch 9/100 |  Train Loss: 0.000917 |  Val Loss: 0.000032
Epoch 10/100 |  Train Loss: 0.000640 |  Val Loss: 0.000031
Epoch 11/100 |  Train Loss: 0.000423 |  Val Loss: 0.000032
Epoch 12/100 |  Train Loss: 0.000686 |  Val Loss: 0.000040
Epoch 13/100 |  Train Loss: 0.000507 |  Val Loss: 0.000051
Epoch 14/100 |  Train Loss: 0.000463 |  Val Loss: 0.000061
Epoch 15/100 |  Train Loss: 0.000415 |  Val Loss: 0.000070
Epoch 16/100 |  Train Loss: 0.000576 |  Val Loss: 0.000071
Epoch 17/100 |  Train Loss: 0.000607 |  Val Loss: 0.000065
Epoch 

In [53]:
def nmse_metric(H_pred, H_true):
    # 그대로 사용 (기존 정의)
    # H_pred, H_true: shape (batch, 32, 64)
    num = torch.sum((H_pred - H_true)**2, dim=[1, 2])
    den = torch.sum((H_true)**2, dim=[1, 2])
    nmse_vals = num / (den + 1e-12)
    return torch.mean(nmse_vals)

# ... 학습이 끝난 뒤 ...


model.eval()
test_nmse = 0.0
count = 0

with torch.no_grad():
    for X_batch, y_batch, masked_batch in test_loader:
        # GPU로 이동
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        masked_batch = masked_batch.to(device)

        # 모델 예측
        pred, _ = model(X_batch, masked_batch)  # shape: (batch, 1, 2048) or (batch, 1, 4096)
        pred = pred.squeeze(1)                 # shape: (batch, 2048)

        # reshape to (batch, 32, 64)
        H_pred = pred.view(-1, 32, 64)         # (batch, 32, 64)
        H_true = y_batch.view(-1, 32, 64)      # (batch, 32, 64)

        nmse_val = nmse_metric(H_pred, H_true)
        test_nmse += nmse_val.item()
        count += 1

test_nmse /= count
print(f"Final Test NMSE: {test_nmse:.6f}")


Final Test NMSE: 141183844352.000000


In [15]:
print(dataset[0][0]['user']['channel'])

[[[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]]


 [[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]]


 [[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]]


 ...


 [[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j

In [53]:
# (예: 사용자 0번, 위치 index 100, scene 0~9까지)
# 같은 사용자의 같은 위치에서 시간에 따라 채널이 변하는지

ue_idx = 0
loc_idx = 100
channel_sequence = []

for scene in dataset:
    channel = scene[ue_idx]['user']['channel']
    h = channel[loc_idx, 0, :, 0]  # shape: (32,)
    channel_sequence.append(h)

channel_sequence = np.stack(channel_sequence)  # shape: (T, 32)


In [65]:
# 입출력 시퀀스
import numpy as np

#batch_size
seq_len = 5

X = []
y = []

for i in range(len(channel_sequence) - seq_len):
    # shape: (seq_len, features)
    # target: (features)
    input_seq = channel_sequence[i:i+seq_len]
    target = channel_sequence[i+seq_len]
    
    X.append(input_seq)
    y.append(target)



X = np.array(X)
y = np.array(y)
# 마스킹 위치 
masked_pos = torch.full((X.shape[0], 1), seq_len - 1, dtype=torch.long)

print("입력 시퀀스 X:", X.shape)
print("예측 대상 y:", y.shape)
print(masked_pos)

입력 시퀀스 X: (5, 5, 32)
예측 대상 y: (5, 32)
tensor([[4],
        [4],
        [4],
        [4],
        [4]])


In [55]:
# 데이터 텐서로 바꾸기
import torch

X_t = torch.tensor(X, dtype=torch.float32, device=device)  # shape: (samples, seq_len, element_length)
y_t = torch.tensor(y, dtype=torch.float32, device=device)  # shape: (samples, element_length)


In [56]:
# DataLoader로 묶기
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_t, y_t)
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)

In [57]:
# LWM 모델 가져오기
from lwm_model import lwm
import torch.nn as nn
import torch.optim as optim

element_length = 64 # 32 imag채널 + 32 real채널

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = lwm(element_length=element_length).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

In [67]:
# 모델 학습
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        seq_len = X_batch.shape[1]
        masked_pos = masked_pos
        
        optimizer.zero_grad()
        pred, _ = model(X_batch, masked_pos)
        pred = pred.squeeze(1)
        
        loss = criterion(pred, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.6f}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (25x32 and 64x64)

In [22]:
print(channel_sequence)

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]]


In [23]:
print(dataset[0][0]['user']['paths'][0]['num_paths'])


0


In [24]:
print(dataset[0][0].keys())


dict_keys(['user', 'basestation', 'location'])


# LWM 동적시나리오 가져와서 분석해보기

In [62]:
channels = []

for scene_idx in range(10):  # scene 0~9
    user_channels = []

    for user in dataset[scene_idx]:  # 각 사용자
        channel = user['user']['channel']  # shape: (1891, 1, 32, 1)
        channel = channel.squeeze(1).squeeze(-1)  # shape: (1891, 32)
        user_channels.append(channel)

    user_channels = np.stack(user_channels)  # (num_users, 1891, 32)
    channels.append(user_channels)

# → channels: (10 scenes, num_users, 1891, 32)
channels = np.stack(channels)             # shape: (10, num_users, 1891, 32)
channels = channels.transpose(1, 0, 2, 3)  # → (num_users, 10, 1891, 32)

# 예: 특정 위치 index만 사용 (예: loc_idx = 100)
loc_idx = 100
channel_data = channels[:, :, loc_idx, :]  # shape: (num_users, 10, 32)

# 데이터 나누기
seq_len = 10
input_ids = torch.tensor(channel_data[:, :seq_len-1, :], dtype=torch.float32)
target_ids = torch.tensor(channel_data[:, seq_len-1, :], dtype=torch.float32)
masked_pos = torch.full((input_ids.shape[0], 1), seq_len-2, dtype=torch.long)


In [63]:
# 학습/테스트 분리
# train 80%, test 20%
split_idx = int(0.8*batch_size)

X_train  = input_ids[:split_idx]
y_train = target_ids[:split_idx]
pos_train = masked_pos[:split_idx]

X_test = input_ids[split_idx:]
y_test = target_ids[split_idx:]
pos_test = masked_pos[split_idx:]

NameError: name 'batch_size' is not defined

In [ ]:
#DataLoader 설정
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train, y_train, pos_train)
test_dataset = TensorDataset(X_test, y_test, pos_test)

train_loader = DataLoader(train_dataset, batch_size=32 , shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
# 학습 파이프라인
from lwm_model import lwm
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = lwm(element_length=element_length).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()


In [ ]:
# 학습 루프
n_epochs = 100


for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    count = 0

    for input_ids_batch, target_ids_batch, masked_pos_batch in train_loader:
        input_ids_batch = input_ids_batch.to(device)
        target_ids_batch = target_ids_batch.to(device)
        masked_pos_batch = masked_pos_batch.to(device)

        optimizer.zero_grad()

        logits, _ = model(input_ids_batch, masked_pos_batch)
        loss = criterion(logits.squeeze(1), target_ids_batch.squeeze(1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        count += 1

    if epoch % 5 == 0:
      avg_loss = total_loss / count
      print(f"Epoch {epoch}: Avg Loss = {avg_loss:.6f}")


In [ ]:
# 예측 후 시각화
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    pred, _ = model(input_ids.to(device), masked_pos.to(device))
    pred = pred.squeeze(1).cpu().numpy()
    target = target_ids.squeeze(1).numpy()

# 예: 사용자 0의 첫 번째 feature
plt.plot(pred[0], label="Predicted")
plt.plot(target[0], label="Actual", linestyle="--")
plt.title("Predicted vs Actual (User 0)")
plt.legend()
plt.grid(True)
plt.show()


# LWM을 가져와서 예측하기

In [60]:
## 데이터 준비 예시

import torch
import numpy as np

# 예시: shape (batch, seq_len, element_length)
# element_length = 채널 real + imag = 예: 64
# seq_len = 예: 10개 시점
# batch_size = 사용자 수

# batch_size가 커질수록 정확도 하락
seq_len = 10 # 총 10개 time step ex(scene 0 ~ 9)
element_length = 64 # 채널 real + imag

# dummy channel data (여기에 실제 채널 데이터 사용)
channels = []  # 전체 scene 채널을 모을 리스트

for scene_idx in range(10):  # scene 0 ~ 9
    user_channels = [] # 해당 scene의 모든 사용자 채널 저장

    # dataset[scene_idx]에서 한 사용자의 딕셔너리는
    # {'user': {...}, 'basestation': {...}} 구조
    for user_dict in dataset[scene_idx]:  # 각 scene의 사용자 순회
        # channel = user_dict['channel']  # ← 이것 때문에 KeyError 발생
        channel = user_dict['user']['channel']  # 실제 데이터 구조에 맞춰 수정

        # (1) shape 확인 후 squeeze
        # 보통 ( something, 1, something, 1 ) 형태라면 이렇게:
        channel = channel.squeeze(1).squeeze(-1)  # → (32,) 형태로 변형
        # (2) 사용자 채널 리스트에 누적
        user_channels.append(channel)

    # scene 하나(10개 중 1개)에 대한 "모든 사용자" 채널을 stack
    channels.append(np.stack(user_channels))  # (user, antenna) shape로 stack

# (scene, user, antenna) → (user, scene, antenna)
channel_data = np.stack(channels, axis=0).transpose(1, 0, 2)  # → (사용자 수, 10, 32)
batch_size = channel_data.shape[0]  # 사용자 수 기반으로 설정

# 아래코드는 dumy 채널 데이터였음
# channel_data = np.random.randn(batch_size, seq_len, element_length).astype(np.float32)

input_ids = torch.tensor(channel_data[:, :seq_len-1, :])       # t=0~(n-2)
target_ids = torch.tensor(channel_data[:, seq_len-1:seq_len, :])  # t=(n-1)
masked_pos = torch.full((batch_size, 1), seq_len-2)  # 마지막 step 위치


SyntaxError: expected ':' (3271643563.py, line 22)

In [29]:
print(type(user))

<class 'dict'>


In [30]:
print(user)

{'user': {'paths': array([{'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
       {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
       {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
       ...,
       {'num_paths': 1, 'DoD_phi': array([-4.67836], dtype=float32), 'DoD_theta': array([90.9207], dtype=float32), 'DoA_phi': array([175.322], dtype=float32), 'DoA_theta': array([89.0793], dtype=float32), 'phase': array([3.80264], dtype=float32), 'ToA': array([1.03828e-06], dtype=float32), 'LoS': array([0.], dtype=float32), 'power': array([4.792699e-10], dtype=float32)},
       {'num_paths': 1, 'DoD_phi': array([-4.67536], dtype=float32), 'DoD_theta': array([90.9201], dtype=float32), 'DoA_phi': array([175.325], dtype=float32), 'DoA_theta': array([89.07

In [32]:
print(dataset[0][0])

{'user': {'paths': array([{'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
       {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
       {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
       ...,
       {'num_paths': 1, 'DoD_phi': array([-4.67836], dtype=float32), 'DoD_theta': array([90.9207], dtype=float32), 'DoA_phi': array([175.322], dtype=float32), 'DoA_theta': array([89.0793], dtype=float32), 'phase': array([3.80264], dtype=float32), 'ToA': array([1.03828e-06], dtype=float32), 'LoS': array([0.], dtype=float32), 'power': array([4.792699e-10], dtype=float32)},
       {'num_paths': 1, 'DoD_phi': array([-4.67536], dtype=float32), 'DoD_theta': array([90.9201], dtype=float32), 'DoA_phi': array([175.325], dtype=float32), 'DoA_theta': array([89.07